In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import math
import numpy as np
import pymongo
import re
import urllib.request as urllib2
import datetime

from time import sleep
from bs4 import BeautifulSoup

def getSoupObj(url):
    sleep(1.5)
    try:
        r = urllib2.urlopen(url)
    except UnicodeEncodeError as e:
        return None
    html = r.read()
    htmltext= html.decode('utf-8')
    return BeautifulSoup(htmltext,'lxml')


## Starting the scrapping

In [0]:
base_url = "https://en.wikipedia.org"
hdr = {'User-Agent': 'Mozilla/5.0'}

### Getting the urls of all players

In [0]:
def getPlayers(playersObj,playersURL,flag):

  groups = playersObj.findAll('div', {'class': 'mw-category-group'})
  if flag==0:
    groups = playersObj.findAll('div', {'class': 'mw-category'})
  links = groups[flag].findAll('a')
  print("Cantidad de players en esta pag: ", len(links))
  for link in links:
    playersURL.append(link.get('href'))

  return

In [0]:

playersURL=[]
nextPageURL= base_url+'/w/index.php?title=Category:Serie_A_players'
flag=2
while True:
  
  req = urllib2.Request(nextPageURL,headers=hdr)
  print(nextPageURL)
  playersObj = getSoupObj(req)
  
  #getting players url from the page
  
  getPlayers(playersObj, playersURL,flag)
  flag=0
  nextPage= playersObj.findAll('a', {'title': 'Category:Serie A players'})
  if(nextPage[1].text=="next page"):
    nextPageURL= base_url + nextPage[1].get('href')
  else:
    print("SE ACABARON LAS PAGS")
    break
print(len(playersURL))

https://en.wikipedia.org/w/index.php?title=Category:Serie_A_players
Cantidad de players en esta pag:  197
https://en.wikipedia.org/w/index.php?title=Category:Serie_A_players&pagefrom=Antunes%2C+Vitorino%0AVitorino+Antunes#mw-pages
Cantidad de players en esta pag:  200
https://en.wikipedia.org/w/index.php?title=Category:Serie_A_players&pagefrom=Bedin%2C+Maurizio%0AMaurizio+Bedin#mw-pages
Cantidad de players en esta pag:  200
https://en.wikipedia.org/w/index.php?title=Category:Serie_A_players&pagefrom=Bombardini%2C+Davide%0ADavide+Bombardini#mw-pages
Cantidad de players en esta pag:  200
https://en.wikipedia.org/w/index.php?title=Category:Serie_A_players&pagefrom=Caldara%2C+Mattia%0AMattia+Caldara#mw-pages
Cantidad de players en esta pag:  200
https://en.wikipedia.org/w/index.php?title=Category:Serie_A_players&pagefrom=Celustka%2C+Ondrej%0AOnd%C5%99ej+%C4%8Cel%C5%AFstka#mw-pages
Cantidad de players en esta pag:  200
https://en.wikipedia.org/w/index.php?title=Category:Serie_A_players&page

In [0]:
matrix=[]
for link in playersURL:
  nextPlayerURL=base_url+link
  req = urllib2.Request(nextPlayerURL,headers=hdr)
  print(nextPlayerURL)
  playerObj = getSoupObj(req)
  
  name = playerObj.find('h1', {'id': 'firstHeading'}).text
  body= playerObj.find('table', {'class': 'infobox vcard'})
  if body==None:
    continue
  rows= body.findAll('tr')
  
  flag=0
  seasons=[]
  teams=[]
  for row in rows:
    header=row.find('th')
    if row.text[:13]=="Senior career":
      flag=1
    elif flag==1:
      flag=2
    elif flag==2:
      if header==None:
        print("se acabaron por pie de nota") #PRINTING THE REASONS WHY ONE PAGE WAS OVER TO FIND ERRORS
        break
      elif header.get('colspan')=='4':
        print("colspan 4")
        break
      years=header.text
      team=row.find('td').text
      if not team.strip():
        print("no team")
        break
      seasons.append(years)
      print(years)
      teams.append(team)
      
  matrix.append(name)
  matrix.append(seasons)
  matrix.append(teams)

print(len(matrix))
#print(teams)

https://en.wikipedia.org/wiki/Mika_Aaltonen
1982–1987
1987–1988
1988
1988–1989
1989–1990
1990–1993
1993–1994
1994
colspan 4
https://en.wikipedia.org/wiki/Rolando_Aarons
2014–
2018
2018
2019
colspan 4
https://en.wikipedia.org/wiki/Roberto_Aballay
1940–1941
1942
1943
1944–1945
1946–1948
1949
1950–1952
1952–1955
1955–1956
se acabaron por pie de nota
https://en.wikipedia.org/wiki/Beniamino_Abate
1983–1985
1985–1990
1990–1991
1991–1994
1994–1995
1995–1997
1997–2000
2000–2001
no team
https://en.wikipedia.org/wiki/Ignazio_Abate
2003–2007
2004–2005
2005–2006
2006–2007
2007–2008
2008–2009
2009–
colspan 4
https://en.wikipedia.org/wiki/Julio_Abbadie
1950–1956
1956–1960
1960–1962
1962–1969
no team
https://en.wikipedia.org/wiki/Matteo_Abbate
2003–2009
2007
2009–2010
2010–2013
2013
2013–2014
2014–2016
2016–2017
2017–2018
se acabaron por pie de nota
https://en.wikipedia.org/wiki/Christian_Abbiati
1994–1998
1995–1996
1998–2016
2005–2006
2006–2007
2007–2008
no team
https://en.wikipedia.org/wiki/Alessan

In [0]:
print(len(matrix)/3) #We divide between three because, its actually a list with this format: (player,years,teams,player,years,teams,player...)

4841.0


In [0]:
print(matrix[1])

['1982–1987', '1987–1988', '1988', '1988–1989', '1989–1990', '1990–1993', '1993–1994', '1994']


In [0]:
matrix[0]

'Mika Aaltonen'

### Saving the list of players,years and teams in a pickle

In [0]:
import pickle

with open('/content/drive/My Drive/outfile', 'wb') as fp:
    pickle.dump(matrix, fp)


In [0]:
#with open ('/content/drive/My Drive/outfile', 'rb') as fp:
#   itemlist = pickle.load(fp)